# 1. Import Required Libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import color
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from glob import glob
import os


# 2. Load Dataset (Positive & Negative Samples)

In [2]:
# Load positive samples (people)
pos_images = glob("C:/Users/XyloCrafts/Desktop/week 13-14/Dataset 1 CCTV/human detection dataset/1/*.png")

# Load negative samples (non-people images)
neg_images = glob("C:/Users/XyloCrafts/Desktop/week 13-14/Dataset 1 CCTV/human detection dataset/0/*.png")


# 3. Extract HOG Features from Images

In [4]:
def extract_hog_features(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (64, 128))  # Resize to standard size
    gray = color.rgb2gray(image)  # Convert to grayscale
    features = hog(gray, 
                   orientations=9, 
                   pixels_per_cell=(8, 8), 
                   cells_per_block=(2, 2), 
                   block_norm='L2-Hys')  # Extract HOG features
    return features


# 4. Prepare Training Data

In [5]:
# Extract features for all images
X = []
y = []

# Process positive images
for img in pos_images:
    X.append(extract_hog_features(img))
    y.append(1)  # Label for "person" is 1

# Process negative images
for img in neg_images:
    X.append(extract_hog_features(img))
    y.append(0)  # Label for "non-person" is 0

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)


# 5. Train SVM Classifier

In [6]:
# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10, random_state=20)

# Train SVM
svm = SVC(kernel='linear')  # Use a linear SVM
svm.fit(X_train, y_train)

# Evaluate Model
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 80.00%


# 6. Detect People in a Test Image

In [7]:
def detect_person(image_path, svm_model):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (64, 128))  # Resize
    gray = color.rgb2gray(image)  # Convert to grayscale
    features = hog(gray, orientations=9, pixels_per_cell=(8, 8), 
                   cells_per_block=(2, 2), block_norm='L2-Hys')  # Extract HOG

    features = np.array(features).reshape(1, -1)  # Reshape for SVM
    prediction = svm_model.predict(features)

    if prediction == 1:
        print("Person Detected ✅")
    else:
        print("No Person Detected ❌")

# Test on a new image
detect_person("cs.png", svm)


Person Detected ✅


In [8]:
import cv2
import numpy as np
from skimage.feature import hog
from skimage import color

def detect_person(image_path, svm_model):
    image = cv2.imread(image_path)
    image_resized = cv2.resize(image, (64, 128))  # Resize for feature extraction
    gray = color.rgb2gray(image_resized)  # Convert to grayscale
    
    # Extract HOG features
    features = hog(gray, orientations=9, pixels_per_cell=(8, 8), 
                   cells_per_block=(2, 2), block_norm='L2-Hys')
    
    features = np.array(features).reshape(1, -1)  # Reshape for SVM
    prediction = svm_model.predict(features)
    
    label = "Person Detected ✅" if prediction == 1 else "No Person Detected ❌"
    color_text = (0, 255, 0) if prediction == 1 else (0, 0, 255)
    
    # Display the result on the image
    output_image = image.copy()
    cv2.putText(output_image, label, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, color_text, 2)
    
    cv2.imshow("Detection Result", output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Test on a new image
detect_person("cs.png", svm)


# 7. Real-Time Person Detection

In [ ]:
cap = cv2.VideoCapture(0)  # Open webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_resized = cv2.resize(frame, (64, 128))
    gray = color.rgb2gray(frame_resized)
    features = hog(gray, orientations=9, pixels_per_cell=(8, 8), 
                   cells_per_block=(2, 2), block_norm='L2-Hys')

    features = np.array(features).reshape(1, -1)
    prediction = svm.predict(features)

    if prediction == 1:
        cv2.putText(frame, "Person Detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Live Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
